In [1]:
# -*- coding: utf-8 -*-
"""XGBoostWB_Electricity
"""
import sys
#Import Libraries
#import tensorflow as tf
import pandas as pd
import numpy as np
import os
#import matplotlib
#import matplotlib.pyplot as plt
import random
# %matplotlib inline
import shutil
from random import shuffle
random.seed(42) 
import itertools

from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from datetime import datetime, timedelta
from sklearn import preprocessing

%load_ext autoreload
%autoreload 2
from mpmf.utils import *



In [2]:
print_hello_world()

Hello, world!


In [3]:
include_features = True
include_motif_information = True

In [4]:
file_name = "electricity.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
data = data[0:1, :]
# data= data[0:10,:]
data = pd.DataFrame(data)
###################################################
num_periods_output = 24  # to predict
num_periods_input = 24  # input


m = num_periods_output - 1
l = 1  # the next l points after the motif

In [5]:
ALL_Test_Data = []
ALL_Test_Prediction = []

# Processing Dataset

## Helper Function

In [6]:
"""## preprocessing"""


def New_preprocessing(TimeSeries):
    Data = []
    start_date = datetime(2012, 1, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])  # adding the electricity value
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])

        # Hide
        # record.append(np.nan)
        # record.append(np.nan)
        # record.append(np.nan)
        # record.append(np.nan)
        # record.append(np.nan)
        # record.append(np.nan)

        # Append Motif information
        # if include_motif_information:
        #     record.extend(df_motif.iloc[i].tolist())
        start_date = start_date + timedelta(hours=1)
        Data.append(record)
    ########## change list of lists to df ################
    headers = [
        "electricity",
        "month",
        "day",
        "hour",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
    # if include_features and include_motif_information:
    #     headers=['electricity','month','day','hour','day_of_week','day_of_year','week_of_year'] + df_motif.columns.tolist()
    # elif include_features and (not include_motif_information):
    #     headers=['electricity','month','day','hour','day_of_week','day_of_year','week_of_year']
    # elif (not include_features) and include_motif_information:
    #     headers=['electricity'] + df_motif.columns.tolist()
    # else:
    #    #  headers=['electricity','month','day','hour','day_of_week','day_of_year','week_of_year'] # default
    #     headers=['electricity']
    Data_df = pd.DataFrame(Data, columns=headers)
    # print(Data_df)
    sub = Data_df.iloc[:, 1:]
    # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
    New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
    Normalized_Data_df = pd.DataFrame(
        np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
    )

    if include_motif_information:
        df_motif = get_top_1_motif(TimeSeries, m, l, True)
        Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif], axis=1)

    # if include_features and include_motif_information:
    #     pass
    # elif include_features and (not include_motif_information):
    #     Normalized_Data_df = Normalized_Data_df.iloc[:, :len(headers)]
    # elif (not include_features) and include_motif_information:
    #     Normalized_Data_df = Normalized_Data_df.iloc[:, [0] + df_motif.columns.tolist()]
    # else:
    #     pass

    #################################################################################################
    # cut training and testing training is 25968
    Train = Normalized_Data_df.iloc[0:25968, :]
    Train = Train.values
    Train = Train.astype("float32")
    #    print('Training length :',len(Train))
    Test = Normalized_Data_df.iloc[(25968 - num_periods_input) :, :]
    Test = Test.values
    Test = Test.astype("float32")
    #    print('Test length :',len(Test))
    # Number_Of_Features=7
    Number_Of_Features = len(Normalized_Data_df.columns)
    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    count = 0
    while next + (num_periods_input) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    # print('Length of y batches :',len(y_batches),' ',num_periods_input,' ',num_periods_output)
    # print(x_batches)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)
    # print('len x_batches ',len(x_batches))
    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + num_periods_input
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)
    # print('len Test',len(Test))
    # print('len xTestbatches',len(x_testbatches))
    # interested_cols = np.r_[0, -4:]
    # x_batches = x_batches[:, :, [0, -1]]
    x_batches = x_batches[:, :, :]
    x_testbatches = x_testbatches[:, :, :]
    # x_testbatches = x_testbatches[:, :, [0, -1]]
    print("len x_batches", x_batches.shape, " len y_batches", y_batches.shape)
    print(
        "len x_testbatches",
        x_testbatches.shape,
        " len y_testbatches",
        y_testbatches.shape,
    )
    return x_batches, y_batches, x_testbatches, y_testbatches

## Main

In [7]:
# TimeSeries=data.iloc[0,:]
# T = TimeSeries

In [8]:
# x_batches, y_batches, x_testbatches, y_testbatches = New_preprocessing(TimeSeries)

In [9]:
# y_testbatches

In [10]:
# df_motif = get_motif_information(TimeSeries, m, l)

In [11]:
# Data=[]
# start_date=datetime(2012, 1, 1,00,00,00) # define start date
# for i in range(0,len(TimeSeries)):
#     record=[]
#     record.append(TimeSeries[i])#adding the electricity value
#     record.append(start_date.month)
#     # print(start_date.month)
#     record.append(start_date.day)
#     record.append(start_date.hour)
#     record.append(start_date.weekday())
#     # print(start_date.weekday())
#     record.append(start_date.timetuple().tm_yday)
#     record.append(start_date.isocalendar()[1])
#     # append all motif columns (no matter how many)
#     record.extend(df_motif.iloc[i].tolist())
#     #print(start_date.month,' ',start_date.day,' ',start_date.hour,' ',start_date.weekday(),' ',start_date.timetuple().tm_yday,' ',start_date.isocalendar()[1])
#     start_date=start_date+ timedelta(hours=1)
#     #print('year',start_date.year,'Month:',start_date.month,' day:',start_date.day,' hour:',start_date.hour)
#     Data.append(record)

In [12]:
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
###################################################
for i in range(0, len(data)):
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(TimeSeries)
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)
    print("Finished processing time series", i + 1, "of", len(data))

# ---------------------shuffle windows  X and target Y together-------------------------------------
combined = list(zip(x_batches_Full, y_batches_Full))
random.shuffle(combined)
shuffled_batch_features, shuffled_batch_y = zip(*combined)

# xgboost part
All_Training_Instances = []

# =============== flatten each training window into Instance =================================
for i in range(0, len(shuffled_batch_features)):
    hold = []
    temp = []
    for j in range(0, len(shuffled_batch_features[i])):
        # **************** to run without features -->comment if else condition (just keep else statement) **************************
        # Fixed
        #   if include_features or include_motif_information:
        #       if j==(len(shuffled_batch_features[i])-1):
        #             hold=np.concatenate((hold, shuffled_batch_features[i][j][:]), axis=None)
        #       else:
        #         hold=np.concatenate((hold, shuffled_batch_features[i][j][0]), axis=None)
        #   else:
        #       hold=np.concatenate((hold, shuffled_batch_features[i][j][0]), axis=None)
        if j == (len(shuffled_batch_features[i]) - 1):
            hold = np.concatenate((hold, shuffled_batch_features[i][j][:]), axis=None)
        else:
            hold = np.concatenate((hold, shuffled_batch_features[i][j][0]), axis=None)
        # hold=np.concatenate((hold, shuffled_batch_features[i][j][:]), axis=None)
    All_Training_Instances.append(hold)


# print(len(All_Training_Instances[0]))

# =============== flatten each testing window into Instance =================================
All_Testing_Instances = []
for i in range(0, len(X_Test_Full)):
    hold = []
    temp = []
    for j in range(0, len(X_Test_Full[i])):
        # **************** to run without features -->comment if else condition (just keep else statement) **************************
        # Fixed
        #   if include_features or include_motif_information:
        #       if j==(len(X_Test_Full[i])-1):
        #           hold=np.concatenate((hold, X_Test_Full[i][j][:]), axis=None)
        #       else:
        #           hold=np.concatenate((hold, X_Test_Full[i][j][0]), axis=None)
        #   else:
        #       hold=np.concatenate((hold, X_Test_Full[i][j][0]), axis=None)
        if j == (len(X_Test_Full[i]) - 1):
            hold = np.concatenate((hold, X_Test_Full[i][j][:]), axis=None)
        else:
            hold = np.concatenate((hold, X_Test_Full[i][j][0]), axis=None)
        # hold=np.concatenate((hold, X_Test_Full[i][j][:]), axis=None)
    All_Testing_Instances.append(hold)

# print(len(All_Testing_Instances[0]))

# =========================== final shape check =========================
All_Testing_Instances = np.reshape(
    All_Testing_Instances, (len(All_Testing_Instances), len(All_Testing_Instances[0]))
)
Y_Test_Full = np.reshape(Y_Test_Full, (len(Y_Test_Full), num_periods_output))

All_Training_Instances = np.reshape(
    All_Training_Instances,
    (len(All_Training_Instances), len(All_Training_Instances[0])),
)
shuffled_batch_y = np.reshape(
    shuffled_batch_y, (len(shuffled_batch_y), num_periods_output)
)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


len x_batches (25921, 24, 11)  len y_batches (25921, 24, 1)
len x_testbatches (7, 24, 11)  len y_testbatches (7, 24, 1)
Finished processing time series 1 of 1


In [13]:
# All_Testing_Instances

# Train

In [ ]:
# =========================== CALLING XGBOOST ===========================
model = xgb.XGBRegressor(
    learning_rate=0.2,
    n_estimators=150,
    max_depth=8,
    min_child_weight=1,
    gamma=0.0,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=1,
    # seed=42,silent=False
    random_state=42,
    verbosity=1,
)

multioutput = MultiOutputRegressor(
    model, n_jobs=1  # ensures sequential fitting, avoids nondeterminism in threading
).fit(All_Training_Instances, shuffled_batch_y)


print("Fitting Done!")

In [ ]:
All_Testing_Instances.shape

(490, 31)

In [ ]:
# ============================== PREDICTION ===============================
prediction = multioutput.predict(All_Testing_Instances)

In [ ]:
# prediction, Y_Test_Full

# Test

In [ ]:
print_prediction_results(prediction, Y_Test_Full)

RMSE:  141.68057
WAPE:  0.09365346
MAE:  52.456245


# Results
- test
```
RMSE:  161.62082
WAPE:  0.10943094
MAE:  61.293365

t =10
RMSE:  67.59057
WAPE:  0.100073926
MAE:  36.478226

without 
RMSE:  68.286255
WAPE:  0.100807734
MAE:  36.74571


RMSE:  139.36037
WAPE:  0.10478689
MAE:  58.69219

no delta + no dist
RMSE:  5.4564867
WAPE:  0.07584218
MAE:  3.7579868

no delta
RMSE:  5.4955373
WAPE:  0.078908354
MAE:  3.9099162

Wih all
RMSE:  5.7024674
WAPE:  0.07927269
MAE:  3.9279687
 
With feature
RMSE:  4.746327
WAPE:  0.06622907
MAE:  3.2816563

With out feature
RMSE:  5.7828765
WAPE:  0.080296926
MAE:  3.9787197
```
- top-k motif family
- top-k motif
- top-1 motif
```
With Original Features + With Motif Information (top-1, l=5)

Without Original Features + With Motif Information (top-1, l=5)

With Original Features + With Motif Information (top-1, l=2)

Without Original Features + With Motif Information (top-1, l=2)

With Original Features + With Motif Information (top-1, l=1)
Witha all
RMSE:  155.0223
WAPE:  0.10833517
MAE:  60.67961

Without Original Features + With Motif Information (top-1, l=1)
```
- Original
```
With Original Features
RMSE:  132.66907
WAPE:  0.09288037
MAE:  52.023224

Without Original Features
RMSE:  136.25429
WAPE:  0.10342441
MAE:  57.92905
```